In [ ]:
from datetime import timedelta, date
import os, time
import matplotlib.colors as colors
from netCDF4 import Dataset
from osgeo import gdal

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)


def HexToRgb(hex_string):
    rgb = colors.hex2color(hex_string)
    r,g,b = tuple([int(255*x) for x in rgb])
    return (r,g,b)

def ProducePng(datestr):
    cmd = "rm -f *.tif*"
    !$cmd
    netcdf = 'NETCDF:"Map_%s.nc":pm25' % (datestr)
    cmd = 'GDAL_NETCDF_BOTTOMUP=NO gdal_translate -a_ullr -180 -90 180 90 -a_nodata -9999 -of GTiff  -b 1 %s %s.tif' % (netcdf, datestr)
    !$cmd
    cmd = 'gdalwarp -s_srs EPSG:4326 -t_srs EPSG:3857 -te -20037508.34 -20037508.34 20037508.34 20037508.34 -ts 512 512 -r cubicspline %s.tif %s-3857.tif' % (datestr, datestr)
    !$cmd
    cmd = 'gdaldem color-relief -alpha %s-3857.tif ramp.txt %s-color-3857.tif' % (datestr, datestr)
    !$cmd
    cmd = "convert %s-color-3857.tif %s.png" % (datestr, datestr)
    !$cmd

In [ ]:
ProducePng("20161230")

In [ ]:
ProducePng("20170108")

In [ ]:
start_date = date(2016, 6, 1)
end_date = date(2017, 3, 20)
for single_date in daterange(start_date, end_date):
    ProducePng(single_date.strftime("%Y%m%d"))


In [ ]:
cmd = "ffmpeg -framerate 30 -pattern_type glob -i '*.png' -c:v libx264 -s:v 512x512 -pix_fmt yuv420p -bf 0 -g 10 out.mp4"
!$cmd

In [ ]:
rootgrp = Dataset("Map_20160608.nc", "r", format="NETCDF4")
lon = rootgrp.variables['longitude']
lat = rootgrp.variables['latitude']
print lon[0:10]
print lon[len(lon)-10: len(lon)]
print lat[0:10]
print lat[len(lat)-10: len(lat)]

In [ ]:
rootgrp.variables['pm25'][23][100][0]

In [ ]:
rootgrp = Dataset("Map_20160609.nc", "r", format="NETCDF4")
lon = rootgrp.variables['longitude']
lat = rootgrp.variables['latitude']
print lon[0:10]
print lon[len(lon)-10: len(lon)]
print lat[0:10]
print lat[len(lat)-10: len(lat)]

In [ ]:
rootgrp = Dataset("Map_20160618.nc", "r", format="NETCDF4")
lon = rootgrp.variables['longitude']
lat = rootgrp.variables['latitude']
print len(lon)
print len(lat)
print lon[0:10]
print lon[len(lon)-10: len(lon)]
print lat[0:10]
print lat[len(lat)-10: len(lat)]


In [ ]:
rootgrp.variables['pm25']

In [ ]:
pm25 = rootgrp.variables['pm25']
dir(pm25)
a = pm25[0][0][0]
dir(a)
a.__class__

In [ ]:
start_date = date(2016, 6, 1)
end_date = date(2017, 3, 20)
for single_date in daterange(start_date, end_date):
    datestr  = single_date.strftime("%Y%m%d")
    ncfilename = "Map_%s.nc" % (datestr)
    if os.path.exists(ncfilename):
        rootgrp = Dataset(ncfilename, "r", format="NETCDF4")
        lon = rootgrp.variables['longitude']
        lat = rootgrp.variables['latitude']
        print "%s %s %s" % (ncfilename, len(lon), len(lat))

In [ ]:
rootgrp = Dataset("Map_20160609.nc", "r", format="NETCDF4")
lon = rootgrp.variables['longitude']
lat = rootgrp.variables['latitude']
print lon[0:10]
print lon[len(lon)-10: len(lon)]
print lat[0:10]
print lat[len(lat)-10: len(lat)]

In [ ]:
def ProducePng2(datestr):
    print datestr
    netcdf = 'NETCDF:"Map_%s.nc":pm25' % (datestr)
    src_ds = gdal.Open(netcdf)
    ulx, xres, xskew, uly, yskew, yres  = src_ds.GetGeoTransform()
    lrx = ulx + (src_ds.RasterXSize * xres)
    lry = uly + (src_ds.RasterYSize * yres)    
    cmd = "rm -f *.tif*"
    !$cmd
    cmd = 'GDAL_NETCDF_BOTTOMUP=NO gdal_translate -q -a_ullr %s %s %s %s -a_nodata -9999 -of GTiff  -b 1 %s %s.tif' % (ulx, lry, lrx, uly, netcdf, datestr)
    !$cmd
#    cmd = 'gdalwarp -q -s_srs EPSG:4326 -t_srs EPSG:3857 -te -20037508.34 -20037508.34 20037508.34 20037508.34 -ts 8192 8192 -r cubicspline %s.tif %s-3857.tif' % (datestr, datestr)
    cmd = 'gdalwarp -q -s_srs EPSG:4326 -t_srs EPSG:3857 -te -20037508.34 -20037508.34 20037508.34 20037508.34 -ts 1024 1024 -r cubicspline %s.tif %s-3857.tif' % (datestr, datestr)
    !$cmd
    cmd = 'gdaldem color-relief -q -alpha %s-3857.tif ramp.txt %s-color-3857.tif' % (datestr, datestr)
    !$cmd
    cmd = "convert %s-color-3857.tif %s.png" % (datestr, datestr)
    !$cmd

In [ ]:
ProducePng2(20170101)


In [ ]:
start_date = date(2016, 6, 1)
end_date = date(2017, 3, 20)
for single_date in daterange(start_date, end_date):
    ProducePng2(single_date.strftime("%Y%m%d"))


In [ ]:
dates = []
start_date = date(2016, 6, 1)
end_date = date(2017, 3, 20)
for single_date in daterange(start_date, end_date):
    datestr  = single_date.strftime("%Y%m%d")
    ncfilename = "Map_%s.nc" % (datestr)
    if os.path.exists(ncfilename):
        ProducePng2(datestr)
        dates.append(single_date.strftime("%Y-%m-%d"))

In [ ]:
src_ds = gdal.Open("NETCDF:Map_20160609.nc:pm25")
ulx, xres, xskew, uly, yskew, yres  = src_ds.GetGeoTransform()
lrx = ulx + (src_ds.RasterXSize * xres)
lry = uly + (src_ds.RasterYSize * yres)


In [ ]:
lrx

In [ ]:
lry

In [ ]:
ulx

In [ ]:
uly

In [ ]:
len(dates)

In [ ]:
dates[0]

In [ ]:
ProducePng2(20170211)


In [ ]:
cmd = "ffmpeg -framerate 30 -pattern_type glob -i 'png/*.png' -c:v libx264 -s:v 512x512 -pix_fmt yuv420p -bf 0 -g 10 out.mp4"
!$cmd

In [ ]:
dates[0]

In [ ]:
dates = []
start_date = date(2016, 6, 1)
end_date = date(2017, 3, 20)
for single_date in daterange(start_date, end_date):
    datestr  = single_date.strftime("%Y%m%d")
    ncfilename = "Map_%s.nc" % (datestr)
    if os.path.exists(ncfilename):
        dates.append('"' + single_date.strftime("%Y-%m-%d") + '"')

In [ ]:
dates[0]

In [ ]:
",".join(dates)